In [10]:
# reload magic
%load_ext autoreload
%autoreload 2
import pandas as pd
from pathlib import Path
import pandas as pd
import tqdm
from top2vec import Top2Vec

import pandas as pd
from pathlib import Path
import pandas as pd
from top2vec import Top2Vec
from sentence_transformers import SentenceTransformer


PATH_TO_MODELS=Path('models')
PATH_TO_OUTDATA=Path('data')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
df=pd.read_parquet(PATH_TO_OUTDATA / 'data_dokdelar.pqt')
# Drop 2014-2016
df=df.loc[~df['rm'].isin(['2014/15','2015/16','2015','2016']),:]
df.head()

,Titel,rm,doktyp,Departement,Utskott,dokdelar
h60310_0,Riksrevisionens rapport om deltagarantal i nya...,2018/19,prop,Arbetsmarknadsdepartementet,AU,Regeringens skrivelse 2018/19:10 Riksrevisione...
h60310_1,Riksrevisionens rapport om deltagarantal i nya...,2018/19,prop,Arbetsmarknadsdepartementet,AU,"funktionsnedsättning, Arbetsträning med handle..."
h60310_2,Riksrevisionens rapport om deltagarantal i nya...,2018/19,prop,Arbetsmarknadsdepartementet,AU,ett viktigt bidrag till detta arbete. I och me...
h60310_3,Riksrevisionens rapport om deltagarantal i nya...,2018/19,prop,Arbetsmarknadsdepartementet,AU,Skr. 2018/19:10 Bilaga D E L T A G A R A N T A...
h60310_4,Riksrevisionens rapport om deltagarantal i nya...,2018/19,prop,Arbetsmarknadsdepartementet,AU,R A N S K N I N G S R A P P O R T F R Å N R I ...


In [3]:
df.shape

(362270, 6)

In [4]:
# load Modell_sbertKB_deldok_2017_22.mod from PATH_TO_MODELS
model = Top2Vec.load(PATH_TO_MODELS / 'Modell_sbertKB_deldok_2017_22.mod')

In [6]:
# index document vectors
model.index_document_vectors()

In [11]:
# Set embedding model
sentence_model = SentenceTransformer('KBLab/sentence-bert-swedish-cased').encode
model.set_embedding_model(sentence_model)

In [7]:
for w, s in zip(*model.similar_words(['arbetslöshet','sysselsättning'],10)):
    print(w,s)

arbetslöshets 0.9742750149488548
sysselsätt 0.9637708920418987
sysselsättnings 0.9635155742923663
anställning 0.959090745525697
anställningar 0.9522519159358616
anställnings 0.9503860012360351
arbetsmarknad 0.9483932955412877
arbetsmark 0.9483547367279711
arbetsmarknaden 0.9451331412538372
arbetskraften 0.9440815764605993


In [8]:
qt="""Socialdemokraterna, Moderaterna, Miljöpartiet de gröna, Centerpartiet 
och Kristdemokraterna slöt den 10 juni 2016 en ramöverenskommelse om 
energipolitiken, den s.k. energiöverenskommelsen. Energiöverenskommelsen utgör en gemensam politisk färdplan för en kontrollerad övergång 
till ett helt förnybart elsystem med nya energipolitiska mål. Med överenskommelsen som grund lämnas i denna proposition förslag om nya energipolitiska mål och regeringens bedömning av inriktningen av energipolitiken. 
Regeringen föreslår att energipolitikens tre grundpelare – försörjningstrygghet, konkurrenskraft och ekologisk hållbarhet – fastställs som övergripande mål för energipolitiken. 
Vidare föreslår regeringen att målet år 2040 ska vara 100 procent förnybar elproduktion. Det är ett mål, inte ett stoppdatum som förbjuder kärnkraft och innebär inte heller en stängning av kärnkraft med politiska beslut. 
Regeringen föreslår också att Sverige år 2030 ska ha 50 procent effektivare energianvändning jämfört med 2005, uttryckt i termer av tillförd 
energi i relation till bruttonationalprodukten (BNP). 
I propositionen presenteras även regeringens bedömning av inriktningen 
av energipolitiken mot bakgrund av energiöverenskommelsen. 
"""

In [14]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('KBLab/sentence-bert-swedish-cased').tokenize

In [12]:
docs, scores, ids = model.query_documents(qt,10,use_index=True, tokenizer=tokenizer)

In [21]:
df.reset_index(inplace=True)

In [26]:
df.head()

,index,Titel,rm,doktyp,Departement,Utskott,dokdelar
0,h60310_0,Riksrevisionens rapport om deltagarantal i nya...,2018/19,prop,Arbetsmarknadsdepartementet,AU,Regeringens skrivelse 2018/19:10 Riksrevisione...
1,h60310_1,Riksrevisionens rapport om deltagarantal i nya...,2018/19,prop,Arbetsmarknadsdepartementet,AU,"funktionsnedsättning, Arbetsträning med handle..."
2,h60310_2,Riksrevisionens rapport om deltagarantal i nya...,2018/19,prop,Arbetsmarknadsdepartementet,AU,ett viktigt bidrag till detta arbete. I och me...
3,h60310_3,Riksrevisionens rapport om deltagarantal i nya...,2018/19,prop,Arbetsmarknadsdepartementet,AU,Skr. 2018/19:10 Bilaga D E L T A G A R A N T A...
4,h60310_4,Riksrevisionens rapport om deltagarantal i nya...,2018/19,prop,Arbetsmarknadsdepartementet,AU,R A N S K N I N G S R A P P O R T F R Å N R I ...


In [24]:
for doc, score, doc_id in zip(docs, scores, ids):
    print(f"Document: {doc_id}, Score: {score}, Title: {df.loc[doc_id,'Titel']}")
    print("-----------")
    print(doc)
    print("-----------")
    print()


Document: 191159, Score: 0.9541310667991638, Title: För en god och jämlik hälsa - En utveckling av det folkhälsopolitiska ramverket
-----------
ur protokoll vid regeringssammanträde den 12 april 2018........ 80 rop. 2017/18:228 1Förslag till riksdagsbeslut Regeringen föreslår att riksdagen godkänner regeringens förslag till 1.övergripande mål för energipolitiken (avsnitt 5.1), 2.mål för andelen förnybar elproduktion (avsnitt 5.2), 3.mål för effektivare energianvändning (avsnitt 5.3), 4.upphävande av riksdagsbindningar avseende riktlinjer för energipolitiken (avsnitt 5.4). Prop. 2017/18:228 2Ärendet och dess beredning I mars 2015 beslutade regeringen att tillsätta en kommitté i form av en parlamentarisk kommission för översyn av energipolitiken (dir. 2015:25). Den s.k. Energikommissionens uppdrag var enligt direktivet att ta fram underlag för en bred överenskommelse om energipolitiken med särskilt fokus på förhållandena för elförsörjningen efter år 2025–2030. Den 10 juni 2016 slöts en r

KeyError: 428819